In [1]:
from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core import Environment
from azureml.core import Dataset, Datastore

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig

In [2]:
from azureml.core.compute import ComputeInstance

In [3]:
workspace = Workspace.from_config()

In [4]:
instance = ComputeTarget(workspace=workspace, name='gpu-v100-low')

In [5]:
dataset = Dataset.get_by_name(workspace, name='recursionbio')

In [6]:
dataset

{
  "registration": {
    "id": "ee33756c-ab30-45ec-adf9-5b93629bbbd8",
    "name": "recursionbio",
    "version": 2,
    "workspace": "Workspace.create(name='bioworks-azureml', subscription_id='73f2271c-beee-4233-b5dd-202b520bf0db', resource_group='mslearn-azureml')"
  }
}

In [7]:
tf_env = Environment.from_conda_specification(name="regbio-tf-env", file_path="conda_env_gpu.yml")

In [8]:
tf_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "regbio-tf-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge",
                "defaults"


In [9]:
train_scr = ScriptRunConfig(
    source_directory='./scripts',
    script='training.py',
    arguments=['--data-path', dataset.as_mount(),
               '--epochs', 2500,
               '--batch', 24,
               '--learning-rate', 0.0001,
               ],
    compute_target=instance,
    environment=tf_env
)

In [10]:
train_scr

In [11]:
run = Experiment(workspace=workspace, name='test-training-run').submit(train_scr)

In [12]:
run.wait_for_completion(show_output=True)

RunId: test-training-run_1638481204_5e49e670
Web View: https://ml.azure.com/runs/test-training-run_1638481204_5e49e670?wsid=/subscriptions/73f2271c-beee-4233-b5dd-202b520bf0db/resourcegroups/mslearn-azureml/workspaces/bioworks-azureml&tid=52603c87-c32e-48b2-b2c4-e366f50b2505
